## Scott Breitbach
### DSC 650, Week 7
### 01-May-2022

In [1]:
# Load libraries
import os
import json
from pathlib import Path
import gzip
import hashlib
import shutil
import pandas as pd
import pygeohash
import s3fs
import uuid
import math
import hashlib

In [2]:
# Set directories
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
if results_dir.exists():
    shutil.rmtree(results_dir)
results_dir.mkdir(parents=True, exist_ok=True)
src_data_path = '/home/jovyan/dsc650/data/processed/openflights/routes.jsonl.gz'

In [3]:
def read_jsonl_data():
    with gzip.open(src_data_path, 'rb') as f:
        records = [json.loads(line) for line in f.readlines()]       
    return records

In [4]:
def flatten_record(record):
    flat_record = dict()
    for key, value in record.items():
        if key in ['airline', 'src_airport', 'dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key = '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            flat_record[key] = value
    
    return flat_record

In [5]:
def create_flattened_dataset():
    records = read_jsonl_data()
    parquet_path = results_dir.joinpath('routes-flattened.parquet')
    return pd.DataFrame.from_records([flatten_record(record) for record in records])

## Assignment 7.1
In this part of the assignment, you will partition a dataset using different strategies. You will use the `routes.parquet` dataset you created in a previous assignment. For this dataset, the key for each route will be the three-letter source airport code concatenated with the three-letter destination airport code and the two-letter airline. For instance, a route from Omaha Eppley Airfield (OMA) to Denver International Airport (DEN) on American Airlines (AA) has a key of `OMADENAA`.

### a. 
Start by loading the dataset from the previous assignment using Pandas's [`read_parquet()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_parquet.html) method. Next, add the concatenated key then using Panda's [`apply()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) method to create a new column called `key`. 

In [6]:
# Load dataframe
df = create_flattened_dataset()
# Assign keys
df['key'] = df['src_airport_iata'].astype(str) + df['dst_airport_iata'].astype(str) + df['airline_iata'].astype(str)

For this part of the example, we will create 16 partitions so that we can compare it to the partitions we create from hashed keys in the next part of the assignment. The partitions are determined by the first letter of the composite key using the following partitions.

In [7]:
partitions = (
    ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
    ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
    ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
    ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

In this case `('A', 'A')` means the folder should contain all of the routes whose composite key starts with `A`. Similarly, `('E', 'F')` should contain routes whose composite key starts with `E` or `F`.

The `results/kv` directory should contain the following folders.

In [8]:
# kv
# ├── kv_key=A
# ├── kv_key=B
# ├── kv_key=C-D
# ├── kv_key=E-F
# ├── kv_key=G-H
# ├── kv_key=I-J
# ├── kv_key=K-L
# ├── kv_key=M
# ├── kv_key=N
# ├── kv_key=O-P
# ├── kv_key=Q-R
# ├── kv_key=S-T
# ├── kv_key=U
# ├── kv_key=V
# ├── kv_key=W-X
# └── kv_key=Y-Z

An easy way to create this directory structure is to create a new key called `kv_key` from the `key` column and use the [`to_parquet()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html) method with `partition_cols=['kv_key']` to save a partitioned dataset.

In [9]:
# Set up dictionary of partitions and kv_keys
partition_dict = {}
for i in partitions:
    if i[0] == i[1]:
        partition_dict[i] = i[0]
    else:
        partition_dict[i] = i[0] + '-' + i[1] 

In [10]:
# Generate kv_key from key
def kv_key_gen(data_key):
    for key, val in partition_dict.items():
        if data_key[0] == key[0] or data_key[0] == key[1]:
            return val
    return ''

In [11]:
# Add kv_key column to df
df['kv_key'] = df['key'].apply(kv_key_gen)

In [12]:
df[['key', 'kv_key']]

,key,kv_key
0,AERKZN2B,A
1,ASFKZN2B,A
2,ASFMRV2B,A
3,CEKKZN2B,C-D
4,CEKOVB2B,C-D
...,...,...
67658,WYAADLZL,W-X
67659,DMEFRUZM,C-D
67660,FRUDMEZM,E-F
67661,FRUOSSZM,E-F


In [13]:
# Partition dataset using kv_keys
df.to_parquet(results_dir.joinpath('kv'), partition_cols=['kv_key'])

### b.
Next, we are going to partition the dataset again, but this time we will partition by the hash value of the key. The following is a function that will create a SHA256 hash of the input key and return a hexadecimal string representation of the hash.

In [14]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

We will partition the data using the first character of the hexadecimal hash. As such, there are 16 possible partitions. Create a new column called `hashed` that is a hashed value of the `key` column. Next, create a partitioned dataset based on the first character of the hashed key and save the results to `results/hash`. The directory should contain the following folders.

In [15]:
# hash
# ├── hash_key=0
# ├── hash_key=1
# ├── hash_key=2
# ├── hash_key=3
# ├── hash_key=4
# ├── hash_key=5
# ├── hash_key=6
# ├── hash_key=7
# ├── hash_key=8
# ├── hash_key=9
# ├── hash_key=A
# ├── hash_key=B
# ├── hash_key=C
# ├── hash_key=D
# ├── hash_key=E

In [16]:
# Add hash column to df
df['hashed'] = df['key'].apply(hash_key)

In [17]:
# Add hash key column to df for partitioning
df['hash_key'] = df['hashed'].str[0]

In [18]:
# Partition dataset using hk_hash
df.to_parquet(results_dir.joinpath('hash'), partition_cols=['hash_key'])

### c.
Finally, we will simulate multiple geographically distributed data centers. For this example, we will assume we have three data centers located in the western, central, and eastern United States. Google lists the [locations of their data centers](https://www.google.com/about/datacenters/locations/) and we will use the following locations for our three data centers.  
**West**
* The Dalles, Oregon
* Latitude: 45.5945645
* Longitude: -121.1786823  

**Central**
* Papillion, NE
* Latitude: 41.1544433
* Longitude: -96.0422378  

**East**
* Loudoun County, Virginia
* Latitude: 39.08344
* Longitude: -77.6497145  

Assume that you have an application that provides routes for each of the source airports and you want to store routes in the data center closest to the source airport. The output folders should look as follows.

In [19]:
# geo
# ├── location=central
# ├── location=east
# └── location=west

In [20]:
# Get geohash location info for data centers
data_centers = dict(
    west = pygeohash.encode(45.5945645, -121.1786823),
    central = pygeohash.encode(41.1544433, -96.0422378),
    east = pygeohash.encode(39.08344, -77.6497145)
)

In [21]:
# Get geohash location info for airports
fn = lambda row: pygeohash.encode(row.src_airport_latitude, 
                                  row.src_airport_longitude)
# Add geohash to dataframe
df['geohash'] = df.apply(fn, axis=1)

In [22]:
def find_closest_datacenter(airport_geohash):
    distance = 20000000
    closest = ''
    for key, val in data_centers.items():
        if pygeohash.geohash_haversine_distance(val, airport_geohash) < distance:
            closest = key
            distance = pygeohash.geohash_haversine_distance(val, airport_geohash)

    return closest

In [23]:
# Add column for closest data center
df['location'] = df['geohash'].apply(find_closest_datacenter)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [24]:
# Partition dataset using closest data center location
df.to_parquet(results_dir.joinpath('geo'), partition_cols=['location'])

### d.
Create a Python function that takes as input a list of keys and the number of partitions and returns a list of keys sorted into the specified number of partitions. The partitions should be roughly equal in size. Furthermore, the partitions should have the property that each partition contains all the keys between the least key in the partition and the greatest key in the partition. In other words, the partitions should be ordered.

In [25]:
# Create a list for testing purposes
some_keys = ['S', 'C', 'O', 'T', 'T',
             'M', 'I', 'C', 'H', 'A', 'E', 'L', 
             'B', 'R', 'E', 'I', 'T', 'B', 'A', 'C', 'H']

In [26]:
def sharded(keys, num_partitions):
    for x in range(0, len(keys), num_partitions):
        chunk = keys[x:num_partitions+x]
        yield(chunk)
        
def balance_partitions(keys, num_partitions):
    keys.sort()
    num_per_group = math.ceil(len(keys) / num_partitions)
    return list(sharded(keys, num_per_group))

In [27]:
balance_partitions(some_keys, 3)

[['A', 'A', 'B', 'B', 'C', 'C', 'C'],
 ['E', 'E', 'H', 'H', 'I', 'I', 'L'],
 ['M', 'O', 'R', 'S', 'T', 'T', 'T']]

Looks good, but let's try it with uneven groups:

In [28]:
# Create a list for testing purposes
some_keys = ['S', 'C', 'O', 'T', 'T', 'Y',
             'M', 'I', 'C', 'H', 'A', 'E', 'L', 
             'B', 'R', 'E', 'I', 'T', 'B', 'A', 'C', 'H']

In [29]:
balance_partitions(some_keys, 3)

[['A', 'A', 'B', 'B', 'C', 'C', 'C', 'E'],
 ['E', 'H', 'H', 'I', 'I', 'L', 'M', 'O'],
 ['R', 'S', 'T', 'T', 'T', 'Y']]

Last group is uneven due to rounding. Let's try to fix that:

In [30]:
def sharded(keys, n):
    # Get the total number of keys
    num_keys = len(keys)
    # Number of partitions left to make
    num_part_left = n   
    # Set indices for start and end of partition chunk
    chunk_start, chunk_end = 0, math.ceil(num_keys/num_part_left)
    # Number of keys left to partition
    num_keys_left = num_keys - chunk_start
    for x in range(0, n):
        # Create partition chunk
        chunk = keys[chunk_start:chunk_end]
        # Reduce remaining partitions by 1
        num_part_left -= 1
        # Set start index to previous end index
        chunk_start = chunk_end
        # Number of keys left to partition
        num_keys_left = num_keys - chunk_end
        # Calculate end index for next chunk
        if num_part_left > 0: # Note: cannot divide by zero
            chunk_end = chunk_start + math.ceil(num_keys_left/(num_part_left))
        else:
            chunk_end = chunk_start + num_keys_left
        yield(chunk)

In [31]:
def balance_partitions(keys, num_partitions):
    # Sort list of keys
    keys.sort()
    # Return list of partitions
    return list(sharded(keys, num_partitions))

In [32]:
balance_partitions(some_keys, 3)

[['A', 'A', 'B', 'B', 'C', 'C', 'C', 'E'],
 ['E', 'H', 'H', 'I', 'I', 'L', 'M'],
 ['O', 'R', 'S', 'T', 'T', 'T', 'Y']]

That looks better, now let's make it work with a column from the dataframe:

In [33]:
def balance_partitions(keys, num_partitions):
    # Sort list of keys
    try: # sort df column input
        sorted_keys = keys.sort_values()
    except: # sort list input
        sorted_keys = keys
        sorted_keys.sort()
    # Return list of partitions
    return list(sharded(sorted_keys, num_partitions))

In [34]:
balance_partitions(some_keys, 3)

[['A', 'A', 'B', 'B', 'C', 'C', 'C', 'E'],
 ['E', 'H', 'H', 'I', 'I', 'L', 'M'],
 ['O', 'R', 'S', 'T', 'T', 'T', 'Y']]

In [35]:
balance_partitions(df['airline_country'], 1000)

[11836    ALASKA
 11879    ALASKA
 11878    ALASKA
 11877    ALASKA
 11876    ALASKA
           ...  
 11882    ALASKA
 11913    ALASKA
 11849    ALASKA
 11847    ALASKA
 11812    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 11811    ALASKA
 11810    ALASKA
 11809    ALASKA
 11808    ALASKA
 11807    ALASKA
           ...  
 11915    ALASKA
 11916    ALASKA
 12013    ALASKA
 12012    ALASKA
 12011    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 12010    ALASKA
 12009    ALASKA
 12008    ALASKA
 12007    ALASKA
 12006    ALASKA
           ...  
 11945    ALASKA
 11944    ALASKA
 11943    ALASKA
 11942    ALASKA
 11941    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 11940    ALASKA
 11939    ALASKA
 11938    ALASKA
 11937    ALASKA
 11936    ALASKA
           ...  
 11612    ALASKA
 11611    ALASKA
 11610    ALASKA
 11609    ALASKA
 11608    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 11607    ALASKA
 11606    ALASKA
 11605    ALAS

The other thing I would like to do if I had the time would be to make it so the partition breaks don't fall in the middle of set of like keys.